In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, transpile, assemble, circuit
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram
import numpy as np

In [2]:
def gcd(a,b):
    if a < b:
        return gcd(b,a)
    if b == 0:
        return a
    return gcd(a%b, b)

In [3]:
def mod_by_rep_square(num, repetitions, modulus):
    for i in range(repetitions):
        num = np.mod(num**2, modulus)
    return num

In [4]:
def exponent_to_factors(guess, exponent, num):
    if r % 2 == 1:
        return (1, num)
    else:
        return (gcd(guess ** exponent + 1, num), gcd(guess**exponent - 1, num))

In [31]:
def classical_add(bits, add):
    """ Creates a Quantum Circuit that increments a register of qubits in the Fourier Basis by the parameter add """
    c_add = QuantumCircuit(bits)
    bin_add = bin(add)[2:]
    while len(bin_add) < bits:
        bin_add = '0' + bin_add
    for cbit in range(len(bin_add)):
        if bin_add[cbit] == '1':
            for qbit in range(bits):
                c_add.p( (np.pi/(2**(cbit - qbit))), qbit)
    return c_add
def classical_add_gate(bits, add):
    c_add = classical_add(bits, add).to_instruction()
    c_add.name = "c_add"
    return c_add
c = classical_add(4, 2)
c.draw()

┌────────┐
q_0: ┤ P(π/4) ├
     ├────────┤
q_1: ┤ P(π/2) ├
     └┬──────┬┘
q_2: ─┤ P(π) ├─
     ┌┴──────┤ 
q_3: ┤ P(2π) ├─
     └───────┘

In [32]:
def quantum_add(n1, n2, bits):
    """ Use QFT, classical_add, and Inverse QFT to add two numbers """
    if n1 > (2**(bits - 2)) or n2 > (2**(bits - 2)):
        raise OverflowError
    bn1 = bin(n1)[2:]
    while len(bn1) < bits:
        bn1 = '0' + bn1
    qr = QuantumRegister(bits, "q")
    qc = QuantumCircuit(qr)
    for bit in range(len(bn1)):
        if bn1[bit] == '1':
            qc.x(qr[bits - bit - 1])
    where = []
    for i in range(len(qr)):
        where.append(qr[i])
    qc.append(circuit.library.QFT(num_qubits = bits, do_swaps = True).to_instruction(), where)
    qc.append(classical_add_gate(bits, n2), where)
    qc.append(circuit.library.QFT(num_qubits = bits, do_swaps = True, inverse = True).to_instruction(), where)
    qc.measure_all()
    return qc

In [39]:
q = quantum_add(1, 4, 4)
q.draw()

┌───┐┌──────┐┌────────┐┌──────┐ ░ ┌─┐         
   q_0: ┤ X ├┤0     ├┤0       ├┤0     ├─░─┤M├─────────
        └───┘│      ││        ││      │ ░ └╥┘┌─┐      
   q_1: ─────┤1     ├┤1       ├┤1     ├─░──╫─┤M├──────
             │  qft ││  c_add ││  qft │ ░  ║ └╥┘┌─┐   
   q_2: ─────┤2     ├┤2       ├┤2     ├─░──╫──╫─┤M├───
             │      ││        ││      │ ░  ║  ║ └╥┘┌─┐
   q_3: ─────┤3     ├┤3       ├┤3     ├─░──╫──╫──╫─┤M├
             └──────┘└────────┘└──────┘ ░  ║  ║  ║ └╥┘
meas: 4/═══════════════════════════════════╩══╩══╩══╩═
                                           0  1  2  3

In [43]:
backend = QasmSimulator()
q_comp = transpile(q, backend)
job = backend.run(q_comp, shots = 10)
result = job.result()
result.get_counts(q_comp)

{'0101': 10}